# EDA

This notebook imports the data and explores the current format including visualizing the data.  

In [1]:
# imports 
import pandas as pd 
import numpy as np 
import re

In [2]:
import os 
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import helpers as h

In [3]:
parent_dir = '../../'

In [4]:
df = pd.read_csv(parent_dir + 'data/weather_madrid_LEMD_1997_2015.csv', parse_dates = ['CET'])

In [5]:
df.head()

,CET,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,...,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,1997-01-01,7.0,4.0,2.0,5.0,3.0,2.0,100.0,95.0,76.0,...,10.0,9.0,4.0,13,6,NaN,0.0,6.0,NaN,229
1,1997-01-02,7.0,3.0,0.0,6.0,3.0,0.0,100.0,92.0,71.0,...,10.0,9.0,4.0,26,8,47.0,0.0,5.0,Rain,143
2,1997-01-03,5.0,3.0,2.0,5.0,1.0,-1.0,100.0,85.0,70.0,...,10.0,10.0,7.0,27,19,NaN,0.0,6.0,Rain-Snow,256
3,1997-01-04,7.0,3.0,-1.0,-2.0,-3.0,-4.0,86.0,63.0,49.0,...,10.0,10.0,10.0,27,19,40.0,0.0,2.0,NaN,284
4,1997-01-05,2.0,0.0,-1.0,2.0,0.0,-3.0,100.0,95.0,86.0,...,10.0,5.0,1.0,14,6,NaN,0.0,7.0,Snow,2


Check data types, number of entries, nulls etc:

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6812 entries, 0 to 6811
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   CET                          6812 non-null   datetime64[ns]
 1   Max TemperatureC             6810 non-null   float64       
 2   Mean TemperatureC            6809 non-null   float64       
 3   Min TemperatureC             6810 non-null   float64       
 4   Dew PointC                   6810 non-null   float64       
 5   MeanDew PointC               6810 non-null   float64       
 6   Min DewpointC                6810 non-null   float64       
 7   Max Humidity                 6810 non-null   float64       
 8    Mean Humidity               6810 non-null   float64       
 9    Min Humidity                6810 non-null   float64       
 10   Max Sea Level PressurehPa   6812 non-null   int64         
 11   Mean Sea Level PressurehPa  6812 non-null 

All datatypes make sense so I don't think we need to make any adjustments right now.  

It looks like some of our column headers have preceding spaces which will get annoying.  There's also variability in how the column headers are named so I'm going to get all the column names in a more useable, pythonic form (all lowercase and underscores instead of spaces and no camel case):

In [7]:
df.columns = list(map(lambda x: h.make_pythonic(x), df.columns))

In [8]:
df.columns

Index(['cet', 'max_temperature_c', 'mean_temperature_c', 'min_temperature_c',
       'dew_point_c', 'mean_dew_point_c', 'min_dewpoint_c', 'max_humidity',
       'mean_humidity', 'min_humidity', 'max_sea_level_pressureh_pa',
       'mean_sea_level_pressureh_pa', 'min_sea_level_pressureh_pa',
       'max_visibility_km', 'mean_visibility_km', 'min_visibilityk_m',
       'max_wind_speed_km/h', 'mean_wind_speed_km/h', 'max_gust_speed_km/h',
       'precipitationmm', 'cloud_cover', 'events', 'wind_dir_degrees'],
      dtype='object')

Let me just manually change the date column and that one.........

In [9]:
df.rename(columns = {'min_visibilityk_m': 'min_visibility_km', 'cet': 'date'}, inplace = True);

In [10]:
df.columns

Index(['date', 'max_temperature_c', 'mean_temperature_c', 'min_temperature_c',
       'dew_point_c', 'mean_dew_point_c', 'min_dewpoint_c', 'max_humidity',
       'mean_humidity', 'min_humidity', 'max_sea_level_pressureh_pa',
       'mean_sea_level_pressureh_pa', 'min_sea_level_pressureh_pa',
       'max_visibility_km', 'mean_visibility_km', 'min_visibility_km',
       'max_wind_speed_km/h', 'mean_wind_speed_km/h', 'max_gust_speed_km/h',
       'precipitationmm', 'cloud_cover', 'events', 'wind_dir_degrees'],
      dtype='object')

Now, let's preview data again:

In [28]:
df.head()

,date,max_temperature_c,mean_temperature_c,min_temperature_c,dew_point_c,mean_dew_point_c,min_dewpoint_c,max_humidity,mean_humidity,min_humidity,...,max_visibility_km,mean_visibility_km,min_visibility_km,max_wind_speed_km/h,mean_wind_speed_km/h,max_gust_speed_km/h,precipitationmm,cloud_cover,events,wind_dir_degrees
0,1997-01-01,7.0,4.0,2.0,5.0,3.0,2.0,100.0,95.0,76.0,...,10.0,9.0,4.0,13,6,NaN,0.0,6.0,NaN,229
1,1997-01-02,7.0,3.0,0.0,6.0,3.0,0.0,100.0,92.0,71.0,...,10.0,9.0,4.0,26,8,47.0,0.0,5.0,Rain,143
2,1997-01-03,5.0,3.0,2.0,5.0,1.0,-1.0,100.0,85.0,70.0,...,10.0,10.0,7.0,27,19,NaN,0.0,6.0,Rain-Snow,256
3,1997-01-04,7.0,3.0,-1.0,-2.0,-3.0,-4.0,86.0,63.0,49.0,...,10.0,10.0,10.0,27,19,40.0,0.0,2.0,NaN,284
4,1997-01-05,2.0,0.0,-1.0,2.0,0.0,-3.0,100.0,95.0,86.0,...,10.0,5.0,1.0,14,6,NaN,0.0,7.0,Snow,2


**Let's see where most of the nan's are:**

In [16]:
df.isna().sum()

date                              0
max_temperature_c                 2
mean_temperature_c                3
min_temperature_c                 2
dew_point_c                       2
mean_dew_point_c                  2
min_dewpoint_c                    2
max_humidity                      2
mean_humidity                     2
min_humidity                      2
max_sea_level_pressureh_pa        0
mean_sea_level_pressureh_pa       0
min_sea_level_pressureh_pa        0
max_visibility_km               940
mean_visibility_km              940
min_visibility_km               940
max_wind_speed_km/h               0
mean_wind_speed_km/h              0
max_gust_speed_km/h            3306
precipitationmm                   0
cloud_cover                    1372
events                         5014
wind_dir_degrees                  0
dtype: int64

So out of 6812 entries, 5014 have missing events.  What are events:

In [25]:
df['events'].unique()

array([nan, 'Rain', 'Rain-Snow', 'Snow', 'Fog', 'Fog-Rain',
       'Rain-Thunderstorm', 'Thunderstorm', 'Rain-Hail-Thunderstorm',
       'Fog-Thunderstorm', 'Tornado', 'Fog-Rain-Thunderstorm',
       'Fog-Rain-Snow', 'Fog-Snow', 'Rain-Snow-Thunderstorm', 'Rain-Hail'],
      dtype=object)

So perhaps 'nan's just signify a 'normal' weather day - i.e. 'no event'.  Makes sense. 

**Check duplicates:**

In [24]:
df.duplicated().sum()

0

**Summary Stats:**

In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
max_temperature_c,6810.0,21.039648,8.867187,0.0,13.0,20.0,29.00,41.0
mean_temperature_c,6809.0,14.658687,7.580461,-3.0,8.0,14.0,21.00,32.0
min_temperature_c,6810.0,8.640529,6.837626,-10.0,3.0,9.0,14.00,28.0
dew_point_c,6810.0,8.120705,4.741067,-12.0,5.0,8.0,12.00,20.0
mean_dew_point_c,6810.0,4.976211,4.654270,-15.0,2.0,6.0,8.00,16.0
min_dewpoint_c,6810.0,1.451248,4.909705,-22.0,-2.0,2.0,5.00,14.0
max_humidity,6810.0,81.139354,17.531839,16.0,68.0,87.0,94.00,100.0
mean_humidity,6810.0,57.971366,19.675744,15.0,41.0,59.0,74.00,100.0
min_humidity,6810.0,34.729369,19.320359,4.0,19.0,32.0,47.75,100.0
max_sea_level_pressureh_pa,6812.0,1020.529360,6.235941,994.0,1017.0,1020.0,1024.00,1047.0
